<a href="https://colab.research.google.com/github/ahmedwalidahmad-debug/car-type-classification/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================
# 1️⃣ Imports
# ======================================
import os, shutil, random
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import kagglehub

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import TopKCategoricalAccuracy


In [2]:
# ======================================
# 2️⃣ Download Dataset
# ======================================
print("Downloading dataset...")
DATASET_PATH = kagglehub.dataset_download("eduardo4jesus/stanford-cars-dataset")
print("Dataset path:", DATASET_PATH)

IMAGES_PATH, ANNOS_PATH = None, None
for root, dirs, files in os.walk(DATASET_PATH):
    if "cars_train_annos.mat" in files:
        ANNOS_PATH = os.path.join(root, "cars_train_annos.mat")
    if os.path.basename(root) == "cars_train":
        IMAGES_PATH = root

print("Images path:", IMAGES_PATH)
print("Annotations path:", ANNOS_PATH)


Using Colab cache for faster access to the 'stanford-cars-dataset' dataset.
Dataset path: /kaggle/input/stanford-cars-dataset
Images path: /kaggle/input/stanford-cars-dataset/cars_train/cars_train
Annotations path: /kaggle/input/stanford-cars-dataset/car_devkit/devkit/cars_train_annos.mat


In [3]:
# ======================================
# 3️⃣ Load Annotations
# ======================================
annos = scipy.io.loadmat(ANNOS_PATH)
annotations = annos["annotations"][0]

class_dict = {}
for ann in annotations:
    img_name = ann[5][0]
    class_id = int(ann[4][0][0])
    class_dict.setdefault(class_id, []).append(img_name)

print("Total classes:", len(class_dict))


Total classes: 196


In [4]:
# ======================================
# 4️⃣ Select Top 40 Classes
# ======================================
# ترتيب الكلاسات حسب عدد الصور
sorted_classes = sorted(class_dict.items(), key=lambda x: len(x[1]), reverse=True)
selected_classes = [cls for cls, imgs in sorted_classes[:40]]

OUTPUT_PATH = "/kaggle/working/cars_40_classes"
os.makedirs(OUTPUT_PATH, exist_ok=True)

missing = 0
for cls in selected_classes:
    class_folder = os.path.join(OUTPUT_PATH, f"class_{cls}")
    os.makedirs(class_folder, exist_ok=True)
    for img in class_dict[cls]:
        src = os.path.join(IMAGES_PATH, img)
        dst = os.path.join(class_folder, img)
        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            missing += 1

print("✅ Created dataset with 40 classes at:", OUTPUT_PATH)
print("⚠️ Missing images:", missing)


✅ Created dataset with 40 classes at: /kaggle/working/cars_40_classes
⚠️ Missing images: 0


In [5]:
# ======================================
# 5️⃣ Build DataFrame
# ======================================
DATA_PATH = OUTPUT_PATH
data = []
for cls in sorted(os.listdir(DATA_PATH)):
    if not cls.startswith("class_"):
        continue
    cls_path = os.path.join(DATA_PATH, cls)
    for img in os.listdir(cls_path):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            data.append({"filename": os.path.join(cls_path, img), "label": cls})

df = pd.DataFrame(data)
print("Total images:", len(df))
print("Total classes:", df["label"].nunique())


Total images: 1847
Total classes: 40


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ======================================
# 6️⃣ Train / Validation / Test Split (70% / 10% / 20%)
# ======================================

# الطريقة البديلة: نفصل test أولًا، ثم val، ثم train
# بس بنفس النتيجة

# أولًا: فصل 20% test
df_temp, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# ثانيًا: من الباقي (80%)، فصل 10% val (يعني test_size=0.1 / 0.8 = 0.125 زي ما كان)
train_df, val_df = train_test_split(
    df_temp,
    test_size=0.125,
    stratify=df_temp["label"],
    random_state=42
)

# أو طريقة تانية مباشرة بدون حسابات: استخدم proportions
# بس sklearn مش بيدعم multi-split مباشر، فهنعملها stepwise زي فوق

IMG_SIZE = 224
BATCH_SIZE = 32

# Data augmentation للـ train فقط
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.25,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)

# Validation و Test بدون augmentation
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Generators
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = test_datagen.flow_from_dataframe(
    test_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("NUM_CLASSES:", NUM_CLASSES)

total_samples = len(df)
print(f"Train samples: {len(train_df)} ({len(train_df)/total_samples*100:.1f}%)")
print(f"Validation samples: {len(val_df)} ({len(val_df)/total_samples*100:.1f}%)")
print(f"Test samples: {len(test_df)} ({len(test_df)/total_samples*100:.1f}%)")

Found 1292 validated image filenames belonging to 40 classes.
Found 185 validated image filenames belonging to 40 classes.
Found 370 validated image filenames belonging to 40 classes.
NUM_CLASSES: 40
Train samples: 1292 (70.0%)
Validation samples: 185 (10.0%)
Test samples: 370 (20.0%)


In [7]:
# ======================================
# 7️⃣ Build Model (ResNet50)
# ======================================
base_model = ResNet50(weights="imagenet", include_top=False,
                      input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", TopKCategoricalAccuracy(k=5, name="top5_accuracy")]
)

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,657,320 (94.06 MB)

 Trainable params: 1,069,608 (4.08 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
# ======================================
# 8️⃣ Training Stage 1 (Feature Extraction)
# ======================================
callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    ReduceLROnPlateau(patience=4, factor=0.2, min_lr=1e-6),
    ModelCheckpoint("resnet50_best_stage1.keras", monitor="val_accuracy", save_best_only=True)
]

history_1 = model.fit(
    train_gen, validation_data=val_gen,
    epochs=25, callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 51s 902ms/step - accuracy: 0.0264 - loss: 4.1579 - top5_accuracy: 0.1394 - val_accuracy: 0.0919 - val_loss: 3.4976 - val_top5_accuracy: 0.2919 - learning_rate: 1.0000e-04
Epoch 2/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 24s 569ms/step - accuracy: 0.0799 - loss: 3.5555 - top5_accuracy: 0.2657 - val_accuracy: 0.2054 - val_loss: 3.2720 - val_top5_accuracy: 0.4162 - learning_rate: 1.0000e-04
Epoch 3/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 24s 571ms/step - accuracy: 0.1362 - loss: 3.3274 - top5_accuracy: 0.3732 - val_accuracy: 0.2595 - val_loss: 3.0834 - val_top5_accuracy: 0.4919 - learning_rate: 1.0000e-04
Epoch 4/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 24s 573ms/step - accuracy: 0.1739 - loss: 3.1418 - top5_accuracy: 0.4250 - val_accuracy: 0.2757 - val_loss: 2.8986 - val_top5_accuracy: 0.5568 - learning_rate: 1.0000e-04
Epoch 5/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 24s 583ms/step - accuracy: 0.2263 - loss: 2.9851 - top5_accuracy: 0.5048 - val_accuracy: 0.3027 - val_loss: 2.7570 - val_top5_

In [9]:
# ======================================
# 9️⃣ Fine-Tuning Stage 2
# ======================================
for layer in base_model.layers[-160:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", TopKCategoricalAccuracy(k=5, name="top5_accuracy")]
)

callbacks_ft = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(patience=5, factor=0.2, min_lr=1e-7),
    ModelCheckpoint("resnet50_best_stage2.keras", monitor="val_accuracy", save_best_only=True)
]

history_2 = model.fit(
    train_gen, validation_data=val_gen,
    epochs=50, callbacks=callbacks_ft
)


Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.4353 - loss: 2.0778 - top5_accuracy: 0.7495 - val_accuracy: 0.5243 - val_loss: 1.7775 - val_top5_accuracy: 0.8270 - learning_rate: 1.0000e-05
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 28s 672ms/step - accuracy: 0.5721 - loss: 1.5228 - top5_accuracy: 0.8782 - val_accuracy: 0.5297 - val_loss: 1.7328 - val_top5_accuracy: 0.8486 - learning_rate: 1.0000e-05
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 31s 752ms/step - accuracy: 0.6450 - loss: 1.3094 - top5_accuracy: 0.9096 - val_accuracy: 0.5514 - val_loss: 1.6051 - val_top5_accuracy: 0.8432 - learning_rate: 1.0000e-05
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 26s 623ms/step - accuracy: 0.6946 - loss: 1.1600 - top5_accuracy: 0.9312 - val_accuracy: 0.5514 - val_loss: 1.5365 - val_top5_accuracy: 0.8757 - learning_rate: 1.0000e-05
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 34s 822ms/step - accuracy: 0.7249 - loss: 1.0402 - top5_accuracy: 0.9299 - val_accuracy: 0.5838 - val_loss: 1.4452 - val_top5_ac